<br><br>

<div style='text-align: center; font-family: "Helvetica Neue", Helvetica, Arial, sans-serif;'>
  <span style='font-size: 30px;'>📊</span>
  <span style='font-weight: bold; font-size: 32px; color: #3AB77A;'>Feature Engineering: Purpose</span>
</div>

<br>

<div style='text-align: center; font-family: "Helvetica Neue", Helvetica, Arial, sans-serif;'>
  <span style='font-weight: bold; font-size: 25px; color: #8ADAB4;'>Contexto</span>
</div>

<p style='text-align: center; font-family: "Helvetica Neue", Helvetica, Arial, sans-serif; font-style: italic; color: #8ADAB4; font-size: 20px;'>
  Extraindo alguma informação da intenção do lead com o empréstimo
</p>

In [1]:
import pandas as pd
import numpy as np
import sys, os
sys.path.insert(0, os.path.abspath(".."))

from transformers import pipeline
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

from utils.plot_utils import plotter as ptz

pd.set_option('display.max_columns', 500)
from utils.eda_utils import (
    get_all_information,
    summary_data,
    summary_data_cat,
    StartEda
)

from utils.model_utils import (
    ModelUtils,
    create_pipeline,
    get_class
)

import warnings
warnings.filterwarnings("ignore")

# Dessa vez utilizando a base bronze
df = pd.read_csv('../data/bronze_dataset.csv')

get_all_information(df=df, column_filter='pre_approved', column_target='sent_to_analysis')

df = df[df['pre_approved'] == 1].drop(['pre_approved'],axis=1)

# Realizando os processos de input de NA baseado na regra identificada anteriormente
control = StartEda(df=df, direct_input_cols=['collateral_debt','informed_restriction'])
df = control.df

object_preprocessing = ModelUtils(df=df, target='sent_to_analysis')
X_train, y_train, X_test, y_test, X_val, y_val = object_preprocessing.train_test_val();

Device set to use cpu


# Analisando a problematica dos dados #
De todos os casos nessa base, reprovados temos 19820
De todos os casos nessa base, pré-aprovado temos 14976
Que realmente aprovamos para a analise de crédito foram 3262
Que realmente reprovamos para a analise de crédito foram 11714


In [2]:
import statistics as sta

moda_informed_purpose = sta.mode(X_train['informed_purpose'])
df['informed_purpose'] = df['informed_purpose'].fillna(moda_informed_purpose)

In [3]:
import re

# Intencionalmente deixei as stopwords para o bart processar
def limpar_texto(texto):
    texto = texto.lower()  # tudo minúsculo
    texto = re.sub(r'[^\w\s]', '', texto)  # remove pontuação
    texto = re.sub(r'\s+', ' ', texto).strip()  # remove espaços duplicados
    return texto

df['purpose_normalized'] = df.apply(lambda x: limpar_texto(x['informed_purpose']),axis=1)
df['purpose_normalized'] = df['purpose_normalized'].replace({
    '':'nada a declarar'
}) 

df_to_go = df[['id','purpose_normalized']]

In [4]:
def get_class_purpose(x):
    if pd.isna(x):
        print('oi')
    labels = ["debt", "investment"]

    result = classifier(x, candidate_labels=labels, multi_label=False)
    print(x, result['labels'][0])
    return result['labels'][0] if result['labels'][0] else 'NaN'

In [5]:
df_to_go['ClassPurpose'] = df_to_go.apply(lambda x: get_class_purpose(x=x['purpose_normalized']),axis=1)

cobrir cheque especial investment
pagar dívidas debt
pagamento de cartão de crédiito debt
comprar um sitio de 20 tarefas debt
pagar dividas debt
para comprar outro carro investment
quitar dividas debt
reforma casa investment
nada a declarar debt
pagar o veiculo debt
investir no meu negocio investment
para investimento investment
quitar dividas debt
investimento investment
para investimentos em consultório investment
debts_payment debt
despesas gerais debt
pagar algumas outras dívidas debt
envestimento investment
para comprar outro carro investment
reforma em moradia debt
fazer investimento no escritorio investment
investimento investment
quitar dividas com juros mais altos debt
pagamentos diversos investment
para investir em um novo negócio food truck investment
para quitar meu veiculo debt
nada a declarar debt
renegociação debt
investimento investment
outros recursos investment
qiuitar algumas contas debt
reforma debt
investimento investment
adapitação para cadeirante no propio veicul